In [10]:
library(conos)
library(Seurat)
library(SeuratDisk)
library(sccore)
library(pagoda2)


Attaching package: ‘pagoda2’


The following objects are masked from ‘package:conos’:

    buildWijMatrix, projectKNNs, sgdBatches




In [3]:
pbmc <- LoadH5Seurat("/home/jmitchel/data/cite_seq_data/pbmc_multimodal.h5seurat")

Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding variable feature information for ADT

Adding miscellaneous information for ADT

Initializing SCT with data

Adding counts for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction apca

Adding cell embeddings for apca

Adding feature loadings for apca

Adding miscellaneous information for apca

Adding reduction aumap

Adding cell embeddings for aumap

Adding miscellaneous information for aumap

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction spca

Adding cell embeddings for spca

Adding feature loadings for spca

Adding miscellaneous information for spca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding reduction wnn.umap

Adding cell embeddings for wnn.umap

Adding miscellaneous information for w

In [11]:
all_donors <- unique(pbmc@meta.data$donor)

In [12]:
# make the conos object
all_d_data <- list()
for (d in all_donors) {
    cells_keep <- rownames(pbmc@meta.data)[pbmc@meta.data$donor==d]
    pbmc_sub <- subset(pbmc,cells=cells_keep)
    all_d_data[[d]] <- pbmc_sub@assays$SCT@counts
}

# turn the list of matrices to list of pagoda2 objects
panel.preprocessed <- lapply(all_d_data, pagoda2::basicP2proc, n.cores=5,
                             min.cells.per.gene=0, n.odgenes=2e3,
                             get.largevis=FALSE, make.geneknn=FALSE)

17205 cells, 20729 genes; normalizing ... 

Using plain model 

Winsorizing ... 

log scale ... 

done.


calculating variance fit ...

 using gam 

928 overdispersed genes ... 928

persisting ... 

done.

running PCA using 2000 OD genes .

.

.

.

 done




creating space of type angular done
adding data ... done
building index ... done
querying ... done


running tSNE using 5 cores:


18135 cells, 20729 genes; normalizing ... 

Using plain model 

Winsorizing ... 

log scale ... 

done.


calculating variance fit ...

 using gam 

1086 overdispersed genes ... 1086

persisting ... 

done.

running PCA using 2000 OD genes .

.

.

.

 done




creating space of type angular done
adding data ... done
building index ... done
querying ... done


running tSNE using 5 cores:


17090 cells, 20729 genes; normalizing ... 

Using plain model 

Winsorizing ... 

log scale ... 

done.


calculating variance fit ...

 using gam 

939 overdispersed genes ... 939

persisting ... 

done.

running PCA using 2000 OD genes .

.

.

.

 done




...

running tSNE using 5 cores:


20742 cells, 20729 genes; normalizing ... 

Using plain model 

Winsorizing ... 

log scale ... 

done.


calculating variance fit ...

 using gam 

1672 overdispersed genes ... 1672

persisting ... 

done.

running PCA using 2000 OD genes .

.

.

.

 done




creating space of type angular done
adding data ... done
building index ... done
querying ... done


running tSNE using 5 cores:


26250 cells, 20729 genes; normalizing ... 

Using plain model 

Winsorizing ... 

log scale ... 

done.


calculating variance fit ...

 using gam 

1681 overdispersed genes ... 1681

persisting ... 

done.

running PCA using 2000 OD genes .

.

.

.

 done




creating space of type angular done
adding data ... done
building index ... done
querying ... done


running tSNE using 5 cores:




In [14]:
con <- conos::Conos$new(panel.preprocessed, n.cores=5)

# build graph
con$buildGraph()

# make umap embedding
con$embedGraph(method="UMAP", min.dist=0.01, spread=15, min.prob.lower=1e-3)



found 0 out of 28 cached PCA space pairs ... 

running 28 additional PCA space pairs 

 done

inter-sample links using mNN 

 done

local pairs 

 done


building graph .



.

done

Convert graph to adjacency list...


Done


Estimate nearest neighbors and commute times...




Estimating hitting distances: 15:30:25.
Done.
Estimating commute distances: 15:31:16.
Hashing adjacency list: 15:31:16.
Done.
Estimating distances: 15:31:28.
Done
Done.
All done!: 15:32:39.


Done


Estimate UMAP embedding...


15:32:40 UMAP embedding parameters a = 0.02659 b = 0.7912

15:32:40 Read 161764 rows and found 1 numeric columns

15:32:41 Commencing smooth kNN distance calibration using 5 threads

15:32:48 Initializing from normalized Laplacian + noise

15:33:13 Commencing optimization for 1000 epochs, with 4583178 positive edges using 5 threads

15:35:46 Optimization finished

Done




In [21]:
## label the cell type annotations
con$findCommunities(method=conos::leiden.community, resolution=1)
cell_assigns <- pbmc@meta.data$celltype.l3
names(cell_assigns) <- rownames(pbmc@meta.data)
con$clusters$leiden$groups <- cell_assigns[names(con$clusters$leiden$groups)]

In [25]:
# plot graph
pdf(file = "/home/jmitchel/figures/for_paper_v2/cite_seq_umap_conos.pdf", useDingbats = FALSE,
    width = 7.5, height = 6.5)
con$plotGraph(alpha=0.1)
dev.off()

png 
  2

In [32]:
# plot graph
png(file = "/home/jmitchel/figures/for_paper_v2/cite_seq_umap_conos.png",
    width = 550, height = 475)
con$plotGraph(alpha=0.1)
dev.off()

png 
  2

In [33]:
saveRDS(con,file='/home/jmitchel/data/cite_seq_data/cite_seq_conos.rds')